## AAL calculation

#### The code is modified to calculate AAL for Turkey and Brush Creek probabilistic modeling
- First, it reads the WSE raster value for each event and extracts corresponding WSE values at each building points.
- It then calculates depth of water by subtracting ground elevation and first floor height from the WSE.
          Depth = WSE - Ground Elev - First Floor Height
- It then reads the excel file containing Depth-damage function (DDF) curves, which has damage percentage values based on the value of depth of water.
- It then also reads the csv file storing the probability weights of all the 96 modeled events.
- The damage percentage value for each event is then multiplied by its correspondiong probability weights.
- Eventually, the damage percentage values for all the events are summed up to come up with a final weighted damage percentage value.
- The weighted damage percentage is then multiplied by the value of structure ($) that gives us the loss of structure in US Dollars.

In [1]:
import os
import glob
import pathlib as pl

import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal

In [2]:
def getTifData(tif_path: str):
    """Read a WSE raster and return the gdal objects"""
    src = gdal.Open(tif_path)
    rb = src.GetRasterBand(1)
    gt = src.GetGeoTransform()
    proj = src.GetProjection()
    return rb, gt, src

In [3]:
def query(x: float, y: float, gt: any, rb: any) -> float:
    """Queries one specific cell in the rasterband given an x, y in the 
    geotransform
    """
    px = int((x-gt[0]) / gt[1])   
    py = int((y-gt[3]) / gt[5])   
    return rb.ReadAsArray(px,py,1,1)[0][0]

In [4]:
def StructureData(gdf_path, gt, rb):
    """Read structures, compute WSE values, and depth values"""

    gdf = gpd.read_file(gdf_path) 
    gdf['wse'] = None       # new field to store extracted wse values
    
    for i, idx in enumerate(gdf.index):
        x = gdf.loc[idx, 'x_sp']
        y = gdf.loc[idx, 'y_sp']
        pixel_value = query(x, y, gt, rb)
        
        # Assigning WSE value to the building GeoDataFrame, if value is -9999 it assigns the same -9999 value
        gdf.at[idx, 'wse'] = -9999 if pixel_value == -9999 else pixel_value

        # Calculating actual depth of water based on wse, ground elevation and foundation height. If the wse value is -9999, the depth is also assigned as -9999
        gdf.at[idx,'depth'] = gdf.at[idx,'wse'] - gdf.at[idx,'ground_elv'] - gdf.at[idx,'found_ht'] if gdf.at[idx,'wse'] != -9999 else -9999
    return gdf

In [5]:
def wseBuildingPts(tif_path, gdf_path):
    """Assigns WSE values to each building points"""
    rb, gt, src = getTifData(tif_path)
    wse_result = StructureData(gdf_path, gt, rb)
    return wse_result

In [6]:
def damage_pct_struct(gdf, ddf_struct):
    """Reads building data (gdf) and depth damage function data for structure (ddf_struct) and computes damage percentage values based on the 
    depth of water for each building points. This function is adjusted to work for Turkey and Brush Creek datasets.
    """
    gdf['dmg_pct_str'] = None
    
    # Looping through each row in the GeoDataFrame
    for idx, row in gdf.iterrows():
        occupancy_type = row['occ_new']
        depth = row['depth'] 

        # Checking if depth is -9999, assign 0 damage and continue
        # This is done, because -9999 is NoData value and there is no water in the waster surface raster. So, assigning zero damage
        if depth == -9999:
            gdf.at[idx, 'dmg_pct_str'] = 0
            continue
    
        damage_row = ddf_struct[ddf_struct['OccuNSI'] == occupancy_type]    # Matching row in building file and ddf excel

        # Extracting depth values from damage DataFrame since the depth are named m1 for -1 and p1 for +1 and so on
        depth_values = np.array([
        float(str(col)[1:]) * (-1 if str(col).startswith('m') else 1) if str(col) != '0' else 0
        for col in damage_row.columns[5:]
        ])
    
        damage_values = damage_row.iloc[0, 5:].values.astype(float)      # Extracting damage values
    
        interpolated_damage = np.interp(depth, depth_values, damage_values)    # Interpolating damage percentage for depth valuies
        gdf.at[idx, 'dmg_pct_str'] = interpolated_damage                     

    return gdf

In [7]:
def damage_pct_contents(gdf, ddf_contents):
    """Reads building data (gdf) and depth damage function data for contents (ddf_contents) and computes damage percentage values based on the 
    depth of water for each building points. This function is adjusted to work for Turkey and Brush Creek datasets. 
    This function is similar to damage_pct_struct.
    """
    gdf['dmg_pct_con'] = None
    
    # Loopinging through each row in the GeoDataFrame
    for idx, row in gdf.iterrows():
        occupancy_type = row['occ_new']
        depth = row['depth'] 

        # Checking if depth is -9999, assign 0 damage and continue
        # Same as above function, -9999 being NoData value
        if depth == -9999:
            gdf.at[idx, 'dmg_pct_con'] = 0
            continue
    
        damage_row = ddf_contents[ddf_contents['OccuNSI'] == occupancy_type]  # Matching row in building file and ddf excel

        # Extracting depth values from damage DataFrame since the depth are named m1 for -1 and p1 for +1 and so on
        depth_values = np.array([
        float(str(col)[1:]) * (-1 if str(col).startswith('m') else 1) if str(col) != '0' else 0
        for col in damage_row.columns[5:]
        ])
    
        damage_values = damage_row.iloc[0, 5:].values.astype(float)         # Extracting damage values
    
        interpolated_damage = np.interp(depth, depth_values, damage_values)  # Interpolating damage percentage values based on depth value
        gdf.at[idx, 'dmg_pct_con'] = interpolated_damage

    return gdf

In [8]:
def AAL_all_wse(tif_dir, bld_file, ddf_struct, ddf_contents, prob_csv):
    """Reads all the WSE tif files, probability weights and calculates 
    overall weighted damage values (AAL) for buildings and contents in a single geodataframe"""

    final_gdf = gpd.read_file(bld_file)
    ddf_struct = pd.read_excel(ddf_struct)
    ddf_contents = pd.read_excel(ddf_contents)
    weights_df = pd.read_csv(weights_csv)
    weights_df.set_index('wse_rasters', inplace=True)

    # Getting all TIFF files from the specified directory
    tif_files = glob.glob(os.path.join(tif_dir, "*.tif"))

    # Initializing a damage sum column
    final_gdf['dmg_sum_str'] = 0.0
    final_gdf['dmg_sum_con'] = 0.0

    for tif_file in tif_files:
        raster_name = os.path.splitext(os.path.basename(tif_file))[0]
        print(f"Processing: {raster_name}")

        weight = weights_df.loc[raster_name, 'weight']

        # Calculating WSE and depth, this adds two columns to the building GeoDataFrame
        wse_depth = wseBuildingPts(tif_file, bld_file)

        wse_depth = damage_pct_struct(wse_depth, ddf_struct)                         
        wse_depth = damage_pct_contents(wse_depth, ddf_contents)                       
        

        wse_depth['wgt_dmg_str'] = wse_depth['dmg_pct_str'] * weight     # damage pct structure * probability weight
        wse_depth['wgt_dmg_con'] = wse_depth['dmg_pct_con'] * weight     
        final_gdf['dmg_sum_str'] += wse_depth['wgt_dmg_str']             # sum of weighted damage percentage values from all WSE rasters into a new gdf
        final_gdf['dmg_sum_con'] += wse_depth['wgt_dmg_con']             # for contents   

    # building loss in USD
    final_gdf['dmg_val_str'] = (final_gdf['dmg_sum_str']/100) * final_gdf['val_struct']
    final_gdf['dmg_val_con'] = (final_gdf['dmg_sum_con']/100) * final_gdf['val_cont']
        
    return final_gdf

In [14]:
#Final AAL calculation using all the functions defined above

root_dir = pl.Path(os.getcwd())

tif_directory = root_dir.parent/'Turkey_wse_rasters_Slop'
bld_file = root_dir.parent/'shps/point_shp/Turkey_point_BldAttrb.shp'
ddf_contents = root_dir.parent/'csv_files/ddf_contents.xlsx'    # Depth damage function (for contents) prepared using HAZUS's FAST tool
ddf_struct = root_dir.parent/'csv_files/ddf_struct.xlsx'        # Depth damage function (for structure) prepared using HAZUS's FAST tool
weights_csv = root_dir.parent/'csv_files/event_weights_TurkeyCreek.csv'

final_AAL = AAL_all_wse(tif_directory, bld_file, ddf_struct, ddf_contents, weights_csv)

Processing: 1000_L_q1
Processing: 1000_L_q2
Processing: 1000_L_q3
Processing: 1000_L_q4
Processing: 1000_U_q1
Processing: 1000_U_q2
Processing: 1000_U_q3
Processing: 1000_U_q4
Processing: 100_L_q1
Processing: 100_L_q2
Processing: 100_L_q3
Processing: 100_L_q4
Processing: 100_U_q1
Processing: 100_U_q2
Processing: 100_U_q3
Processing: 100_U_q4
Processing: 10_L_q1
Processing: 10_L_q2
Processing: 10_L_q3
Processing: 10_L_q4
Processing: 10_U_q1
Processing: 10_U_q2
Processing: 10_U_q3
Processing: 10_U_q4
Processing: 2000_L_q1
Processing: 2000_L_q2
Processing: 2000_L_q3
Processing: 2000_L_q4
Processing: 2000_U_q1
Processing: 2000_U_q2
Processing: 2000_U_q3
Processing: 2000_U_q4
Processing: 200_L_q1
Processing: 200_L_q2
Processing: 200_L_q3
Processing: 200_L_q4
Processing: 200_U_q1
Processing: 200_U_q2
Processing: 200_U_q3
Processing: 200_U_q4
Processing: 25_L_q1
Processing: 25_L_q2
Processing: 25_L_q3
Processing: 25_L_q4
Processing: 25_U_q1
Processing: 25_U_q2
Processing: 25_U_q3
Processing: 

In [15]:
#Summing the structure damage and contents damage value to come up with total damage value
final_AAL['dmg_total'] = final_AAL['dmg_val_str'] + final_AAL['dmg_val_con']

In [16]:
#Saving the final GeoDataFrame consisting of final AAL values

output_path_AAL = root_dir.parent/'outputs/aal_out/AAL_TurkeyCreek1.shp'
final_AAL.to_file(output_path_AAL, driver='ESRI Shapefile')

C:\Users\dneupane\AppData\Local\Temp\ipykernel_23080\279639826.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_AAL.to_file(output_path_AAL, driver='ESRI Shapefile')


In [17]:
#Converting crs of final_AAL shapefiles (from state plane to EPSG:4326 to create heatmaps)
aal_shp_state = gpd.read_file(output_path_AAL)
aal_shp_crs = aal_shp_state.to_crs("EPSG:4326")
aal_shp_crs["lon"] = aal_shp_crs.geometry.x
aal_shp_crs["lat"] = aal_shp_crs.geometry.y

output_AAL_crs_path = root_dir.parent/'outputs/aal_out/AAL_TurkeyCreek_crs1.shp'
aal_shp_crs.to_file(output_AAL_crs_path)

In [18]:
#saving as csv
output_csv_path = root_dir.parent/'outputs/AAL_csv_crs1.csv'
aal_shp_crs.to_csv(output_csv_path, index=False)

In [13]:
final_AAL

,MAXELEVFT,MINELEVFT,AVGELEVFT,GNDELEVFT,AID,occtype,found_type,found_ht,val_struct,val_cont,...,Shape_Leng,Shape_Area,x_sp,y_sp,occ_new,geometry,dmg_sum_str,dmg_sum_con,dmg_val_str,dmg_val_con
0,1008,1007.5,1007.5,1005,0,RES1-1SNB,C,1.5,21511.000,20583.0000,...,27.343777,45.128023,2.256047e+06,267078.941174,RES1-1SNB,POINT (2256046.815 267078.941),0.118752,2.66589,25.544798,548.720239
1,1080,1066.9,1072.6,1059,10,RES1-1SWB,B,2.0,200124.846,100062.4230,...,198.638037,1893.290412,2.244923e+06,261502.451571,RES1-1SWB,POINT (2244922.638 261502.452),0.0,0.0,0.0,0.0
2,1063,1057.6,1060.5,1055,11,RES1-1SNB,C,1.5,21511.000,20583.0000,...,52.605331,166.329367,2.245495e+06,256105.366619,RES1-1SNB,POINT (2245495.086 256105.367),0.000996,2.912104,0.214181,599.398431
3,1113,1075.3,1087.6,1071,12,RES1-1SWB,B,2.0,151603.048,75801.5241,...,164.618438,1534.654325,2.247886e+06,248863.960040,RES1-1SWB,POINT (2247886.115 248863.960),0.0,0.0,0.0,0.0
4,1047,1042.4,1045.3,1033,14,RES1-1SNB,C,1.5,157381.720,78690.8602,...,141.102713,1154.055308,2.243756e+06,262306.347305,RES1-1SNB,POINT (2243755.548 262306.347),0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28824,1058,1031.3,1050.3,1019,51389,RES3C,B,0.5,1012446.000,753195.0000,...,430.502079,7037.521406,2.242327e+06,267978.052668,RES3C-B,POINT (2242327.442 267978.053),0.0,0.0,0.0,0.0
28825,1058,1031.3,1050.3,1019,51390,RES3C,B,0.5,1012446.000,753195.0000,...,430.502079,7037.521406,2.242327e+06,267978.052668,RES3C-B,POINT (2242327.442 267978.053),0.0,0.0,0.0,0.0
28826,1033,994.2,1001.5,983,51391,RES1-1SWB,B,2.0,165506.194,82753.0970,...,148.634880,1303.344231,2.251549e+06,256370.160633,RES1-1SWB,POINT (2251549.095 256370.161),0.660316,0.006149,1092.863461,5.088176
28827,995,962.6,980,960,51392,RES1-1SWB,B,2.0,180937.516,90468.7580,...,208.675029,2058.490846,2.250005e+06,271028.564860,RES1-1SWB,POINT (2250004.779 271028.565),0.002299,0.003263,4.159014,2.951897


### End